# Using the Vertex AI PaLM API to Generate and Ideate

# Core code
Let's define some variables that will be used throughout this notebook.

These are the GCP Project ID `project_id`, the Model name `model_name` which is any name you prefer, and finally the Dataset name `dataset_name`.
The dataset needs to exist in the same Project as `project_id` and you'll need appropriate access to create and delete.

In [10]:
from typing import Union
import sys

import os
import io
import json
import base64
import requests
import concurrent.futures
import time
import http.client
import typing
import urllib.request

import numpy as np
import pandas as pd

import vertexai
import vertexai.language_models as language_models
import vertexai.preview.language_models as language_models_preview
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

from google.cloud import aiplatform
from google.cloud import documentai
from google.cloud.documentai_v1 import Document
from google.cloud import storage
from google.cloud import bigquery

from IPython.display import display, Markdown, Latex

print("Vertex AI version: " + str(aiplatform.__version__))

Vertex AI version: 1.41.0


## Setup

In [2]:
PROJECT_ID = 'mg-ce-demos'
REGION = 'us-central1'


In [3]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)
aiplatform.init(project = PROJECT_ID, location = REGION)


## Gemini

### language

In [10]:
def gemini_generate(prompt):
    result = []
    model = GenerativeModel("gemini-pro")
    responses = model.generate_content(
        [prompt],
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.5,
            "top_p": 1
        },
        stream=True,
    )
  
    for response in responses:
        result.append(response.candidates[0].content.parts[0].text)

    result = ''.join(result)
    return result

In [29]:
prompt = """Write a short story about a dog playing in the snow"""

result_txt = gemini_generate(prompt)

In [30]:
print(result_txt)

In the heart of a quaint little town, where snowflakes danced and painted the world in shades of white, lived a dog named Buddy. Buddy, with his wagging tail and boundless energy, found endless joy in the arrival of winter.

One crisp morning, as the sun peeked through the clouds, Buddy's excitement reached new heights. He watched from the window as the snow glistened and sparkled, calling him to an adventure he couldn't resist.

With a bark of delight, Buddy dashed out the door, his paws sinking into the soft, powdery snow. He reveled in the sensation, feeling the cold beneath his fur and the crunch beneath his feet.

As he explored the snow-covered streets, Buddy discovered a winter wonderland that was all his own. He chased snowflakes as they twirled and spun, creating a flurry of white magic around him.

He rolled and tumbled in the snow, leaving behind trails of doggy joy. His laughter echoed through the quiet streets as he slid down snowy slopes and jumped over snow-covered obsta

### multimodal - image and text

In [17]:
# create helper function
def load_image_from_url(image_url: str) -> Image:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return Image.from_bytes(image_bytes)

In [47]:
def gemini_generate_multi_w_image(prompt_list):
    result = []
    model = GenerativeModel("gemini-pro-vision")
    responses = model.generate_content(
        prompt_list, # prompt_list format is [image, prompt, image, prompt, ...]
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.4,
            "top_p": 1,
            "top_k": 32
        },
        stream=True,
    )
  
    for response in responses:
        result.append(response.candidates[0].content.parts[0].text)

    result = ''.join(result)
    return result

In [48]:
# Load images from Cloud Storage URI
landmark1 = load_image_from_url("https://storage.googleapis.com/cloud-samples-data/vertex-ai/llm/prompts/landmark1.png")
landmark2 = load_image_from_url("https://storage.googleapis.com/cloud-samples-data/vertex-ai/llm/prompts/landmark2.png")
landmark3 = load_image_from_url("https://storage.googleapis.com/cloud-samples-data/vertex-ai/llm/prompts/landmark3.png")

# for local images
## source_image = Image.load_from_file(location='./gen-img1.png')

# create multimodal prompt
prompt_data_img = [landmark1, "city: Rome, Landmark: the Colosseum", 
               landmark2, "city: Beijing, Landmark: Forbidden City",
               landmark3, ]

In [36]:
result_img = gemini_generate_multi_w_image(prompt_data_img)

In [37]:
print(result_img)

 city: Rio de Janeiro, Landmark: Christ the Redeemer


### multimodal - video and text

In [40]:
def gemini_generate_multi_w_vid(prompt_list):
    model = GenerativeModel("gemini-pro-vision")
    # Generate text
    response = model.generate_content(prompt_list) #prompt_list format is [prompt, video_data]
    
    return response.text

In [41]:
video_data = Part.from_uri("gs://cloud-samples-data/video/animals.mp4", mime_type="video/mp4")

prompt_data_vid = ["""What is in the video?""", video_data]

In [42]:
result_vid = gemini_generate_multi_w_vid(prompt_data_vid)

In [43]:
print(result_vid)

 The video is an advertisement for the movie Zootopia. It features a tiger, a sloth, and an otter. The tiger is shown stalking the sloth, but the sloth is saved by the otter. The video is set in a rainforest, and the animals are all shown in their natural habitats. The video is narrated by a human, who provides information about the animals and the rainforest.


### function calling

In [5]:
gem_model = GenerativeModel("gemini-pro")

In [7]:
response = gem_model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information and cannot provide the current exchange rate. For the most up-to-date information, please refer to a reliable source such as a currency converter or a financial news website.


In [8]:
user_prompt = "What's the exchange rate from euros to US dollars today?"

response = gem_model.generate_content("""
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object.
""".format(user_prompt=user_prompt))
print(response.text)

```
{
  "from_currency": "euros",
  "to_currency": "US dollars"
}
```


In [9]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.text

'{"amount":1.0,"base":"EUR","date":"2024-02-29","rates":{"AUD":1.6684,"BGN":1.9558,"BRL":5.4054,"CAD":1.4719,"CHF":0.9534,"CNY":7.7888,"CZK":25.363,"DKK":7.454,"GBP":0.85655,"HKD":8.4735,"HUF":393.48,"IDR":17038,"ILS":3.8736,"INR":89.75,"ISK":149.3,"JPY":162.53,"KRW":1447.43,"MXN":18.4969,"MYR":5.1375,"NOK":11.492,"NZD":1.7807,"PHP":60.858,"PLN":4.3208,"RON":4.9706,"SEK":11.215,"SGD":1.457,"THB":38.909,"TRY":33.807,"USD":1.0826,"ZAR":20.901}}'

In [11]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

In [12]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

In [13]:
prompt = """What is the exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = gem_model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [15]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
    }
  }
}



In [16]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'from': 'AUD', 'date': 'latest', 'to': 'SEK'}

In [17]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"amount":1.0,"base":"AUD","date":"2024-02-29","rates":{"SEK":6.722}}'

In [18]:
response = gem_model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

'The exchange rate from Australian dollars to Swedish krona on 2024-02-29 is 1 AUD = 6.722 SEK.\nTherefore, 500 Australian dollars is worth 6.722 * 500 = 3361 Swedish krona.'

## Ideate

In [4]:
textgen_model_32k = language_models_preview.TextGenerationModel.from_pretrained('text-bison-32k')
textgen_model = language_models.TextGenerationModel.from_pretrained('text-bison@latest')

In [5]:
# Provide some context
pre_text = "Pretend you're a marketing specialist, "

# Ask the LLM
text = "Generate a social media post for a new manufacturing robot product"

prompt_ideate = pre_text+text

In [7]:
display(Markdown('## Prompt:'))
print(prompt_ideate)
display(Markdown('## Response:'))

# Send prompt to LLM
display(Markdown(str(textgen_model.predict(
   (prompt_ideate),
    max_output_tokens=1024,
    temperature=0.4,
    top_p=0.8,
    top_k=40,
).text)))

## Prompt:

Pretend you're a marketing specialist, Generate a social media post for a new manufacturing robot product


## Response:

 Introducing the new manufacturing robot product - the ultimate solution for increased productivity and efficiency in your production line. With its state-of-the-art technology and user-friendly interface, this robot is designed to revolutionise the manufacturing industry. Say goodbye to manual labour and hello to a new era of automated production. #ManufacturingRobot #Automation #Productivity #Efficiency #Manufacturing

## Extract 

In [18]:
# Provide some context
pre_text_extract = """Extract the technical specifications from the text below in a JSON format.

Text: type is Mechanical Joint Pipe, pipe size is 3, pipe thickness from .25 to .4, outside diameter is 3.96, bell weight is 11, bolts gasket weight is 7
JSON: {
  \"product_type\":\"Mechanical Joint Pipe\",
  \"size_in\":\"3\",
  \"thickness_in\": [\".25\", \".4\"],
  \"out_diameter_in\":\"3.96\",
  \"bell_weight_lb\":\"11\",
  \"bolts_gasket_weight_lb\":\"7\"
}

Text: type is Mechanical Joint Pipe, pipe size is 4, pipe thickness from .26 to .41, outside diameter is 4.80, bell weight is 16, bolts gasket weight is 10
JSON: {
  \"product_type\":\"Mechanical Joint Pipe\",
  \"size_in\":\"4\",
  \"thickness_in\": [\".26\", \".41\"],
  \"out_diameter_in\":\"4.8\",
  \"bell_weight_lb\":\"16\",
  \"bolts_gasket_weight_lb\":\"10\"
}

"""

# Ask the LLM
text_extract = """Text: type is Mechanical Joint Pipe, pipe size is 5, bolts gasket weight is 12, outside diameter is 5.7, pipe thickness from .28 to .45, bell weight is 18
JSON:
"""

prompt_extract = pre_text_extract+text_extract
#print(prompt_extract)

In [19]:
display(Markdown(str(textgen_model.predict(
   (prompt_extract),
    max_output_tokens=1024,
    temperature=0.4,
    top_p=0.8,
    top_k=40,
).text)))

 {
  "product_type":"Mechanical Joint Pipe",
  "size_in":"5",
  "thickness_in": [".28", ".45"],
  "out_diameter_in":"5.7",
  "bell_weight_lb":"18",
  "bolts_gasket_weight_lb":"12"
}

## Summarize

## Now we can stick it behind a UI

In [56]:
def extractor_bot(text):
    pre_text = """Extract the technical specifications from the text below in a JSON format.

    Text: type is Mechanical Joint Pipe, pipe size is 3, pipe thickness from .25 to .4, outside diameter is 3.96, bell weight is 11, bolts gasket weight is 7
    JSON: {
      \"product_type\":\"Mechanical Joint Pipe\",
      \"size_in\":\"3\",
      \"thickness_in\": [\".25\", \".4\"],
      \"out_diameter_in\":\"3.96\",
      \"bell_weight_lb\":\"11\",
      \"bolts_gasket_weight_lb\":\"7\"
    }
    
    Text: type is Mechanical Joint Pipe, pipe size is 4, pipe thickness from .26 to .41, outside diameter is 4.80, bell weight is 16, bolts gasket weight is 10
    JSON: {
      \"product_type\":\"Mechanical Joint Pipe\",
      \"size_in\":\"4\",
      \"thickness_in\": [\".26\", \".41\"],
      \"out_diameter_in\":\"4.8\",
      \"bell_weight_lb\":\"16\",
      \"bolts_gasket_weight_lb\":\"10\"
    }

    Text: type is Mechanical Joint Pipe, pipe size is 4, pipe thickness from .26 to .41, bell weight is 16, bolts gasket weight is 10
    JSON: {
      \"product_type\":\"Mechanical Joint Pipe\",
      \"size_in\":\"4\",
      \"thickness_in\": [\".26\", \".41\"],
      \"out_diameter_in\":,
      \"bell_weight_lb\":\"16\",
      \"bolts_gasket_weight_lb\":\"10\"
    }
    
    """
    prompt = f"""{pre_text}
    
    Text: {text}
    JSON:
    """
    
    result = textgen_model.predict(
        prompt,
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    ).text

    result_json = json.loads(result)
    
    return result, result_json

In [ ]:
def marketing_bot(text):
    pre_text = "Pretend you're a marketing specialist, "
    prompt = pre_text + text
    result = textgen_model.predict(
        prompt,
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    
    return prompt, result

In [58]:
import gradio as gr
import json

with gr.Blocks() as demo:
    gr.Markdown(
    """
    ## Pocket Engineer Extractor
    """)
    with gr.Row():
        input_text = gr.Textbox(label="Input Text", value="type is Mechanical Joint Pipe, pipe size is 5, bolts gasket weight is 12, outside diameter is 5.7, pipe thickness from .28 to .45, bell weight is 18")
        
    with gr.Row():
        generate = gr.Button("Generate Response")
        
    with gr.Row():
        label3 = gr.Textbox(label="Response generated by LLM")

    with gr.Row():
        label4 = gr.Textbox(label="JSON output")

    generate.click(extractor_bot, input_text, [label3, label4])
demo.launch(share=False, debug=False)

Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
